## Dijkstra Algorithm Implementation using Heap

According to the traditional implementation of Dijkstra Algorithm, its worst case running time is $O(mn)$ with adjacency list, where $m=|E|$ and $n=|V|$. As the traditional method, we have two sets of vertices; $v\in X$ for $v$ are vertices whose the shortest distances from $s$ are already determined, and $w\in V-X$ for $w$ are vertices whose shortest distances from $s$ are not yet determined. We will use `Min-Heap` to store vertices in $V-X$ according to their distances to the nodes in $X$. Here are the steps to implement Dijkstra Algorithm using Heap.

0. Intialize each node in $V-X$ to have $l_{min}=\infty$ 
1. For each node in $V-X$, determine the incoming edge whose length is minimum. Store this value as $l_{min}$.
2. Insert each node in $V-X$ to `Min-Heap` according to their $l_{min}$ values.
3. Remove the minimum node (root) $w$ from the heap. 
4. Move $w$ to $X$
5. Repeat 1-4 until $V-X$ is empty.

The worst case running time for Dijkstra Algorithm using Heap is $O(m\log n)$.


#### Min-Heap Implementation 

In [266]:
class MinHeap:

    def __init__(self, maxSize):
        self.maxSize = maxSize
        self.arr = [None]*maxSize
        self.heapSize = 0

    def heapify(self, i):
        l = 2*i + 1
        r = 2*i + 2
        minimum = i
        if l < self.heapSize and self.arr[l].d < self.arr[i].d:
            minimum = l
        if r < self.heapSize and self.arr[r].d < self.arr[minimum].d:
            minimum = r
        if minimum != i:
            tmp = self.arr[i]
            self.arr[i] = self.arr[minimum]
            self.arr[minimum] = tmp
            self.heapify(minimum)

    def insert(self, x):
        if self.heapSize == self.maxSize:
            print("Overflow: Cannot insert any more node.")
            return
        
        self.heapSize += 1
        i = self.heapSize - 1
        self.arr[i] = x
        parent = (i - 1)//2
        while i != 0 and self.arr[parent].d > self.arr[i].d:
            tmp = self.arr[i]
            self.arr[i] = self.arr[parent]
            self.arr[parent] = tmp
            i = parent
            parent = (i - 1)//2

    def removeMin(self):
        if self.heapSize <= 0:
            return None
        if self.heapSize == 1:
            self.heapSize = 0
            minimum = self.arr[0]
            self.arr[0] = None
            return minimum
        
        minimum = self.arr[0]
        self.arr[0] = self.arr[self.heapSize - 1]
        self.arr[self.heapSize - 1] = None
        self.heapSize -= 1
        self.heapify(0)
        return minimum
    
    # Increases value of key at
    # index 'i' to new_val.
    def increaseKey(self, i, newVal):
        self.arr[i].d = newVal
        parent = (i - 1)//2
        while i != 0 and self.arr[parent].d > self.arr[i].d:
            tmp = self.arr[i]
            self.arr[i] = self.arr[parent]
            self.arr[parent] = tmp
            i = parent
            parent = (i - 1)//2
    
    def deleteKey(self, i):
        # It increases the value of the key
        # to infinity and then removes
        # the maximum value.
        self.increaseKey(i, 0)
        self.removeMin()
    
    def updateHeap(self, w):
        # update l_min for each node connected to w
        # print("Before update d")
        # self.printHeap()
        for i in range(self.heapSize):
            if w.id in self.arr[i].inedges:
                # print("Node {} has incoimng edge from node {}".format(self.arr[i].id, w.id))
                v = self.arr[i]
                tmp_d = v.d
                self.deleteKey(i)
                # print("Delete {}".format(v.id))
                # self.printHeap()
                v.d = tmp_d
                if v.d > (w.d + v.inedges[w.id]):
                     v.d = w.d + v.inedges[w.id]
                     v.v_min = w.id  
                   
                self.insert(v)
                # print("Re-insert {}".format(v.id))
                # self.printHeap()
        # print("After update d")
        # self.printHeap()
        
        return


    def getMin(self):
        return self.arr[0]
    
    def printHeap(self):
        heap = ''
        for i in range(self.heapSize):
            heap += "{}:{}, ".format(self.arr[i].id, self.arr[i].d)
        print(heap)

#### Vertice Class

In [267]:
class Vertice:

    def __init__(self, id):
        self.id = id        # label of the vertice
        self.d = int(1e6)       # shortest distance from src
        self.v_min = None
        self.inedges = {}   # store of incoming edge i.e., vertice: length
        self.outedges = {}  # store of outgoing edge i.e., vertice: length
        self.path = [] 

    def __str__(self):
        return str(self.id)

#### Dijkstra Algorithm

In [268]:
def dijkstra(G):
    s = G[1]
    s.d = 0
    s.path = [s.id]
    X = [s]
    VX = [v for v in G.values()]
    VX.remove(s)
    
    while VX:
        min_w = None
        min_d = int(1e6)
        for w in VX:
            for v in X:
                if v.id in w.inedges:
                    # print("Edge {}-{}".format(v.id, w.id, )) 
                    if v.d + w.inedges[v.id] < min_d:
                        min_d = v.d + w.inedges[v.id]
                        min_w = w
                        
        if min_w:
            min_w.d = min_d
            # print("Remove {} length {}".format(min_w, min_w.d))
            VX.remove(min_w)
            X.append(min_w)
            

In [269]:
def dijkstra_heap(G):
    # Assume vertice 1 is src
    s = G[1]
    s.d = 0
    s.path.append(s.id)
    # Initialize X with s
    X = [s]
    VX = [v for v in G.values()]
    VX.remove(s)
    heap = MinHeap(len(G))
    # determine l_min for each vertice in VX having connection from s
    # then insert to heap
    for v in VX:
        if s.id in v.inedges:
            v.d = s.d + v.inedges[s.id]
            v.v_min = s.id
        heap.insert(v)
    
    # maintain heap structure
    # heap.heapify(0)

    while VX:
        # Print heap to debug
        # print(heap.getMin())
        w = heap.removeMin()
        # print(w.__dict__)
        w.path = G[w.v_min].path + [w.id]
        # print("Remove {} length {} path {}".format(w, w.d, w.path))
        heap.updateHeap(w)
        VX.remove(w)
        X.append(w)
        
            

#### Load adjacency list

In [270]:
with open('dijkstraData.txt') as f:
    lines = f.readlines()

# Initialize Graph
G = {i:Vertice(i) for i in range(1,len(lines)+1)}
for line in lines:
    data = line.rstrip().split()
    v = int(data[0])
    for i in range(1, len(data)):
        w, l = data[i].split(',')
        G[v].outedges[int(w)] = int(l)
        G[int(w)].inedges[v] = int(l)

dijkstra_heap(G)
nodes = [7,37,59,82,99,115,133,165,188,197]
# nodes = [i for i in range(1,11)]
for node in nodes:
    print("Node {} distance {}".format(node, G[node].d))

Node 7 distance 2599
Node 37 distance 2610
Node 59 distance 2947
Node 82 distance 2052
Node 99 distance 2367
Node 115 distance 2399
Node 133 distance 2029
Node 165 distance 2442
Node 188 distance 2505
Node 197 distance 3068


**Answer:**

- Node 7 distance 2599
- Node 37 distance 2610
- Node 59 distance 2947
- Node 82 distance 2052
- Node 99 distance 2367
- Node 115 distance 2399
- Node 133 distance 2029
- Node 165 distance 2442
- Node 188 distance 2505
- Node 197 distance 3068

In [271]:
result = []
for node in nodes:
    result.append(G[node].d)
print(",".join(map(str, result)))

2599,2610,2947,2052,2367,2399,2029,2442,2505,3068
